<a href="https://colab.research.google.com/github/HenryWatson11/WaveDetectorInator3000/blob/main/WaveDetetorModelMK2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import matplotlib
import keras
matplotlib.use("Agg")
from keras.preprocessing.image import ImageDataGenerator
from matplotlib import pyplot as plt
import numpy as np
import random
import os
from matplotlib import pyplot as plt



In [ ]:
!pip install piexif

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

import os
os.listdir("./drive/MyDrive/Wave Detector Images")

Mounted at /content/drive


['Test', 'Train']

In [ ]:
from keras.layers import Dense, Activation, Flatten

In [ ]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense


base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the pre-trained layers
for layer in base_model.layers:
    layer.trainable = False

# Data augmentation and normalization
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

# Flow training images in batches of 32 using train_datagen generator
train_generator = train_datagen.flow_from_directory('./drive/MyDrive/Wave Detector Images/Train', target_size=(224, 224), batch_size=32, class_mode='categorical')

# Build CNN model
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(224, 224, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(4, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(train_generator, epochs=10)

# Evaluate on the test set
test_generator = test_datagen.flow_from_directory('./drive/MyDrive/Wave Detector Images/Test', target_size=(224, 224), batch_size=32, class_mode='categorical')
test_loss, test_acc = model.evaluate(test_generator)
print(f'Test accuracy: {test_acc}')

Found 93 images belonging to 4 classes.
Epoch 1/10
3/3 [==============================] - 11s 3s/step - loss: 37.9237 - accuracy: 0.3226
Epoch 2/10
3/3 [==============================] - 6s 2s/step - loss: 28.2926 - accuracy: 0.4516
Epoch 3/10
3/3 [==============================] - 9s 2s/step - loss: 25.6079 - accuracy: 0.3441
Epoch 4/10
3/3 [==============================] - 7s 2s/step - loss: 8.4680 - accuracy: 0.3011
Epoch 5/10
3/3 [==============================] - 10s 2s/step - loss: 9.3765 - accuracy: 0.4839
Epoch 6/10
3/3 [==============================] - 7s 2s/step - loss: 4.2956 - accuracy: 0.4839
Epoch 7/10
3/3 [==============================] - 8s 2s/step - loss: 2.9941 - accuracy: 0.4516
Epoch 8/10
3/3 [==============================] - 9s 3s/step - loss: 1.2682 - accuracy: 0.5269
Epoch 9/10
3/3 [==============================] - 6s 2s/step - loss: 1.3348 - accuracy: 0.4301
Epoch 10/10
3/3 [==============================] - 7s 2s/step - loss: 1.1233 - accuracy: 0.4731
Foun

In [ ]:
from keras.preprocessing.image import load_img, img_to_array
from keras.applications.vgg16 import preprocess_input
from keras.applications.vgg16 import VGG16
from keras.models import Model
from pickle import dump
import os

# Function to extract features using VGG16 model
def extract_features(directory):
    model = VGG16()
    model = Model(inputs=model.inputs, outputs=model.layers[-2].output)
    features = dict()

    for filename in os.listdir(directory):
        path = os.path.join(directory, filename)
        try:
            #print(f"Processing image: {filename}")

            image = load_img(path, target_size=(224, 224))
            image = img_to_array(image)
            image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
            image = preprocess_input(image)
            feature = model.predict(image, verbose=0)
            image_id = filename.split('.')[0]
            features[image_id] = feature
        except Exception as e:
            print(f"Error processing image {filename}: {str(e)}")

    return features

# Extract features for the training dataset
train_features1 = extract_features('./drive/MyDrive/Wave Detector Images/Train/1')
train_features3 = extract_features('./drive/MyDrive/Wave Detector Images/Train/3')
train_features5 = extract_features('./drive/MyDrive/Wave Detector Images/Train/5')
train_features7 = extract_features('./drive/MyDrive/Wave Detector Images/Train/7')
train_features9 = extract_features('./drive/MyDrive/Wave Detector Images/Train/9')


In [ ]:
features_array = np.array(train_features1)

In [ ]:
print(features_array.shape)

()


In [ ]:
test_features1 = extract_features('./drive/MyDrive/Wave Detector Images/Test/1')
test_features3 = extract_features('./drive/MyDrive/Wave Detector Images/Test/3')
test_features5 = extract_features('./drive/MyDrive/Wave Detector Images/Test/5')
test_features7 = extract_features('./drive/MyDrive/Wave Detector Images/Test/7')

AttributeError: ignored

In [ ]:
print(test_features1)

{'1_2022-01-23_Biarritz': array([[0.15669212, 0.44638646, 1.153813  , ..., 0.        , 0.        ,
        0.        ]], dtype=float32), '1_2022-07-12_Biarritz': array([[0.7326284 , 0.        , 0.33225232, ..., 0.26889426, 0.        ,
        0.        ]], dtype=float32)}


In [ ]:
X_train = []
y_train = []
x = 1
grand_features = [train_features1.values(), train_features3.values(), train_features5.values(), train_features7.values()]
for gf in grand_features:
  for features in gf:
    X_train.append(features)
    y_train.append(x)
  x = x + 2
X_train = np.array(X_train)
y_train = np.array(y_train)

X_train = X_train.reshape((93, 4096))

In [ ]:
print(X_train)

[[0.         0.54783565 0.80448085 ... 1.4006131  0.         0.        ]
 [0.         1.2917182  1.090375   ... 2.3968258  0.         0.        ]
 [0.         0.         1.9170331  ... 0.86083233 0.         0.        ]
 ...
 [0.56703514 0.         1.3879606  ... 0.         0.         1.1612408 ]
 [0.3669699  0.         0.03711605 ... 0.         0.         0.        ]
 [1.7006311  0.         0.5967729  ... 0.         0.         0.        ]]


In [ ]:
print(y_train)

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 5 5 5 5 5 5 5 5 5 5 5 5 5 5 7 7]


In [ ]:
print(X_train.shape)

(93, 4096)


In [ ]:
X_test = []
y_test = []
x = 1
grand_features_test = [test_features1.values, test_features3.values, test_features5.values, test_features7.values]
for gf2 in grand_features_test:
  for featurez in gf2:
    X_test.append(featurez)
    y_test.append(x)
  x = x + 2
X_test = np.array(X_test)
y_test = np.array(y_test)



AttributeError: ignored

In [ ]:
print(y_test)

[1 1 3 3 5 5 7 7]


In [ ]:
y_test = [1, 1, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 5, 5, 5, 5, 5, 5, 5, 5, 7, 7]

In [ ]:
from keras.models import Model
from keras.applications.inception_v3 import InceptionV3

model2 = InceptionV3(include_top = False, input_shape = (200, 200, 3))
model2 = Model(inputs = model.inputs, outputs = model.layers[-1].output)

x_train_FE = model.predict(train_generator)
x_train_FE = x_train_FE.reshape(x_train_FE.shape[0], -1)

print(x_train_FE.shape)

x_test_FE = model.predict(test_generator)
x_test_FE = x_test_FE.reshape(x_test_FE.shape[0], -1)

lr_model2 = LogisticRegression()
lr_model2.fit(x_train_FE, y_train)
predictions = lr_model2.predict(x_test_FE)
from sklearn.metrics import accuracy_score
accuracy_score(predictions, y_test)

3/3 [==============================] - 3s 743ms/step
(93, 4)
1/1 [==============================] - 0s 418ms/step


0.4782608695652174

In [ ]:
# Save features to a file
dump(train_features1, open('train_features1.pkl', 'wb'))
dump(train_features3, open('train_features3.pkl', 'wb'))
dump(train_features5, open('train_features5.pkl', 'wb'))
dump(train_features7, open('train_features7.pkl', 'wb'))
dump(train_features9, open('train_features9.pkl', 'wb'))

In [ ]:
from pickle import load
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.preprocessing.image import ImageDataGenerator

# Load extracted features for each class
#train_features1 = load(open('train_features1.pkl', 'rb'))
#train_features3 = load(open('train_features3.pkl', 'rb'))
#train_features5 = load(open('train_features5.pkl', 'rb'))
#train_features7 = load(open('train_features7.pkl', 'rb'))
#train_features9 = load(open('train_features9.pkl', 'rb'))

# Combine features and labels
#X_train = np.concatenate([train_features1, train_features3, train_features5, train_features7, train_features9])
#y_train = np.array([1]*len(train_features1) + [3]*len(train_features3) + [5]*len(train_features5) +
                   #[7]*len(train_features7) + [9]*len(train_features9))

from keras.models import Sequential
from keras.layers import Dense

# One-hot encode the labels
y_train_one_hot = np.zeros((len(y_train), np.max(y_train)+1))
y_train_one_hot[np.arange(len(y_train)), y_train] = 1

# Build CNN model
model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(X_train.shape[1],)))  # Corrected input_shape
model.add(Dense(np.max(y_train)+1, activation='softmax'))  # Adjust units for the number of classes

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train_one_hot, epochs=10)

TypeError: ignored

In [ ]:
from sklearn.linear_model import LogisticRegression
lr_model = LogisticRegression()
lr_model.fit(X_train, y_train)
predictions = lr_model.predict(X_test)
sum(predictions == y_test) / len(y_test)



/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.0